В этом файле рассматривались различные методы кластеризации для первоначального выбора. Например, Affinity Propogation совсем не разделяет кластеры.    

In [2]:
import pandas
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
#from matplotlib import pyplot as plt
import seaborn as sns
matplotlib.use('TkAgg')
import math

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [3]:
np.__version__

'1.21.5'

In [4]:
# Считываем данные эксперимента
AllExp = pandas.read_csv('datachanged/AllExpC')

In [5]:
AllExp.head()

,Unnamed: 0,num_of_fam,j,X(J),Y(J),E(J),R,cnt,ER,sum_energy,lg_r,lg_e_r
0,0,1,1,0.2513,-1.0370,20.896,1.067015,13,22.296342,145.8,0.028170,1.348234
1,1,1,2,0.6003,-0.5482,13.749,0.812947,13,11.177213,145.8,-0.089938,1.048334
2,2,1,3,0.3081,-1.0631,11.535,1.106846,13,12.767464,145.8,0.044087,1.106105
3,3,1,4,0.6618,-0.5704,6.129,0.873691,13,5.354850,145.8,-0.058642,0.728747
4,4,1,5,0.5843,-0.5611,8.619,0.810086,13,6.982133,145.8,-0.091469,0.843988


In [6]:
# Считываем данные модели
AllMc0 = pandas.read_csv('datachanged/AllMc0C')

In [7]:
AllMc0.head()

,Unnamed: 0,num_of_family,j,X(J),Y(J),E(J),H(J),A0,R,cnt,ER,sum_energy,lg_r,lg_e_r
0,0,1,1,-3.013189,-7.227429,4.611031,1255.250,1,7.830392,26,36.106180,370.199507,0.893784,1.557582
1,1,1,2,-1.310529,-2.563248,12.047700,1545.054,1,2.878841,26,34.683415,370.199507,0.459218,1.540122
2,2,1,3,-2.178048,-2.144561,5.401844,3044.520,1,3.056638,26,16.511481,370.199507,0.485244,1.217786
3,3,1,4,-0.965465,-0.082880,5.325506,4715.523,1,0.969016,26,5.160501,370.199507,-0.013669,0.712692
4,4,1,5,-1.621645,0.079540,15.935060,3044.520,1,1.623595,26,25.872076,370.199507,0.210478,1.412831


In [8]:
N_num = set(AllMc0[' num_of_family']) # Число пленок в эксперименте.
# Создаем список из всех датафреймов для удобства
List_of_Frames = []
for i in N_num: 
    OneFamily = pandas.DataFrame( AllMc0.loc[lambda AllMc0: AllMc0[' num_of_family'] == i, :] ).copy()
    List_of_Frames.append(OneFamily)

In [9]:
OneFamily = pandas.DataFrame( AllMc0.loc[lambda AllMc0: AllMc0[' num_of_family'] == 1, :] ).copy()

In [10]:
# Функция вычисления матрицы расстояния для одного датафрейма
def Matrix_of_Distance(X, Y, E, label):
    Distance =[]
    
    for i in range(len(X)):
        Distance.append([])
        for j in range(len(Y)):
            
            if label == 1:
                Coef = 1
            elif label==2:
                Coef = math.sqrt(E[i]*E[j])
            elif label ==3:
                Coef=( E[i]*E[j])/(E[i] + E[j])
            
            r = Coef*(math.sqrt( math.pow((X[i]-X[j]),2) + math.pow((Y[i]-Y[j]),2) )  )
            #r = abs( (E[i]**2)*(X[i]**2 + Y[i]**2) -  (E[j]**2)*(X[j]**2 + Y[j]**2))
            Distance[i].append(r)
    
    return pandas.DataFrame(Distance)

In [11]:
# Функция, которая преобразует выданные алгоритмом "шумовые" точки в точки, обозначаемые одиночным кластером.

def noise_cluster(labels):
    
    if -1 not in labels:
        print('there is no -1')
        return labels
    
    coef = len(set(labels))-1
    for i in range(len(labels)):  
        if labels[i] == -1:
            coef+=1
            labels[i] = labels[i]+coef
    return labels

In [12]:
# Функция подсчета значений коэффициентов

def CoefPFI(df, num, cluster_column, hlabel_column):
    P = 0
    E = 0
    DomH = [] #доминирующие высоты в кластерах
    N_c = len(df)
    
    labels = df[cluster_column]
    
    n_clusters_ = len(set(labels))
    
    for i in range(n_clusters_):
        
        value = df[df[cluster_column]==i][hlabel_column].value_counts()

        n_d = value[:1].values
        n_c = len(df[df[cluster_column]==i][hlabel_column])
        
        
        P_i = n_d[0]/n_c
        P = P + P_i
        
        x = value[:1].index[0]
        n_t = len(df[df[hlabel_column]==x])
        
        E_i = n_d[0]/n_t
        E = E + E_i
        DomH.append(x)
    
    purity = P/n_clusters_ #1
    
    splitting = len(set(DomH))/n_clusters_ #2
    
    integrity = len(set(DomH))/len(set(df[hlabel_column])) #3
 
    efficiency =  E/n_clusters_ #4
    
    print(f'P = {round(purity,3)}, S = {round(splitting,3)}, I = {round(integrity, 3)}, E = {round(efficiency, 3)}')
   
    
    result = [ num, round(purity, 3), round(splitting,3), round(integrity, 3), round(efficiency,3)] 
    mean = np.mean(result[1:])
    var = np.var(result[1:])
    compose = (purity+efficiency)*(integrity+splitting)/4
    
    result = result + [mean]
    result = result +[var]
    
    
    print(f'Mean: {round(mean, 3)}, compose: {round( compose , 3)}')
    
    return 
    #return CoefS

In [75]:
k = 518
X = Matrix_of_Distance(List_of_Frames[k]['X(J)'].values, List_of_Frames[k]['Y(J)'].values, List_of_Frames[k]['E(J)'].values, 2)

In [76]:
#List_of_Frames[k]

## hdbscan
https://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html  

In [77]:
#!pip install hdbscan --no-cache-dir --no-binary :all:
# !pip uninstall hdbscan
import hdbscan 

In [106]:
clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
                            gen_min_span_tree=False, leaf_size=1, min_cluster_size=2, min_samples=None, p=None)
clusterer.fit(X)
clusterer.labels_

array([-1,  0, -1,  2,  1,  2,  1,  1,  2,  1, -1,  0,  0,  2, -1,  5, -1,
       -1, -1, -1, -1,  5,  5,  3, -1,  3, -1,  6,  5,  4,  5,  6,  5,  5,
        4,  5,  5,  4,  6,  6,  5,  4, -1, -1, -1, -1,  6, -1,  6,  6,  6,
        6], dtype=int64)

In [107]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clusterer.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,19
11249,536,2,654.4922,1,0
11250,536,3,654.4922,1,8
11292,536,45,890.9880,2,20
11290,536,43,1324.9020,3,18
11260,536,13,1862.9310,4,0
11252,536,5,2290.8640,5,1
11254,536,7,2290.8640,5,1
11255,536,8,2290.8640,5,1
11257,536,10,2290.8640,5,1


In [108]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.818, S = 0.933, I = 0.609, E = 0.85
Mean: 0.802, compose: 0.643


In [109]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

-0.042857987347894075

Визуально кажется, что алгоритм разделяет достаточно плохо, хотя значения коэффициентов качества кластеризации получились высокими. Возвращаемые значения "шума" превращаем в отдельные кластеры.  
Не принимает на вход параметров кластеризации.  

## AffinityPropagation
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation

In [110]:
from sklearn.cluster import AffinityPropagation
clustering = AffinityPropagation(damping = 0.7, random_state=5).fit(X)
clustering.labels_

array([ 0,  4,  0,  1,  2,  1,  2,  2,  1,  2,  3,  4,  4, 12,  5,  7, 12,
       12, 12,  5,  5, 12,  7,  5, 12,  5,  6, 12,  7,  5,  7, 12,  7,  7,
        5,  7,  7,  5, 12, 12,  7,  5,  8,  9, 10, 11, 12,  5, 12, 12, 12,
       12], dtype=int64)

In [111]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,9
11249,536,2,654.4922,1,4
11250,536,3,654.4922,1,0
11292,536,45,890.9880,2,10
11290,536,43,1324.9020,3,8
11260,536,13,1862.9310,4,4
11252,536,5,2290.8640,5,2
11254,536,7,2290.8640,5,2
11255,536,8,2290.8640,5,2
11257,536,10,2290.8640,5,2


In [112]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.831, S = 0.667, I = 0.769, E = 0.602
Mean: 0.717, compose: 0.515


In [113]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

0.06752460203250411

damping - ?  
Не принимает на вход параметров кластеризации.  

## AgglomerativeClustering
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering

In [114]:
from sklearn.cluster import AgglomerativeClustering
#linkage: average, complete, single.
thr = np.percentile(List_of_Frames[0]['ER'], 50)
clustering = AgglomerativeClustering(n_clusters=None, affinity = 'precomputed', distance_threshold = thr, linkage = 'average').fit(X)
clustering.labels_

array([12,  9,  8,  2,  3,  2,  3,  3,  2,  3, 11, 13,  5, 14,  1,  0,  1,
        0,  1,  1,  0,  0,  0,  0,  1,  0, 10,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0, 15,  6,  4,  7,  0,  0,  0,  0,  0,
        0], dtype=int64)

In [115]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,6
11249,536,2,654.4922,1,9
11250,536,3,654.4922,1,8
11292,536,45,890.9880,2,4
11290,536,43,1324.9020,3,15
11260,536,13,1862.9310,4,5
11252,536,5,2290.8640,5,3
11254,536,7,2290.8640,5,3
11255,536,8,2290.8640,5,3
11257,536,10,2290.8640,5,3


In [116]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.916, S = 0.867, I = 0.812, E = 0.82
Mean: 0.854, compose: 0.729


In [117]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

0.3394495607548803

## MeanShift
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.MeanShift.html#sklearn.cluster.MeanShift

In [119]:
from sklearn.cluster import MeanShift
thr = np.percentile(List_of_Frames[0]['ER'], 50)
clustering = MeanShift(bandwidth=thr).fit(X)
clustering.labels_

array([45, 23, 44, 19,  1, 20,  9,  1, 26, 13, 21, 10,  7, 43, 25,  3, 42,
       39, 41, 28, 16, 35,  3, 22, 40, 24,  4, 36,  0, 17,  2, 30,  2,  0,
       15, 27,  0, 18, 31, 37,  0, 12,  6, 11,  8,  5, 32, 14, 29, 33, 34,
       38], dtype=int64)

In [120]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,11
11249,536,2,654.4922,1,23
11250,536,3,654.4922,1,44
11292,536,45,890.9880,2,8
11290,536,43,1324.9020,3,6
11260,536,13,1862.9310,4,7
11252,536,5,2290.8640,5,1
11254,536,7,2290.8640,5,9
11255,536,8,2290.8640,5,1
11257,536,10,2290.8640,5,13


In [121]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.743, S = 1.0, I = 0.326, E = 1.0
Mean: 0.767, compose: 0.578


In [122]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

-0.005476173234870165

Похож на dbscan.  

## OPTICS
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.OPTICS.html#sklearn.cluster.OPTICS

In [123]:
from sklearn.cluster import OPTICS
clustering = OPTICS(min_samples=2).fit(X)
clustering.labels_

array([ 0, 13,  0, 10, 11, 10, 11, 11, 10, 11, 13, 13, 13, -1, -1,  4, -1,
       -1,  1, -1,  8, -1,  4,  8,  1,  8, -1,  3,  6,  9,  7,  3,  7,  5,
        9, -1,  6,  9,  3,  3,  5,  9, 14, 12, 12, 14,  2, -1, -1,  2,  3,
        2])

In [124]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,12
11249,536,2,654.4922,1,13
11250,536,3,654.4922,1,0
11292,536,45,890.9880,2,12
11290,536,43,1324.9020,3,14
11260,536,13,1862.9310,4,13
11252,536,5,2290.8640,5,11
11254,536,7,2290.8640,5,11
11255,536,8,2290.8640,5,11
11257,536,10,2290.8640,5,11


In [125]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.804, S = 0.667, I = 0.4, E = 0.6
Mean: 0.618, compose: 0.374


In [126]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

0.08803744279713474

Нет параметров.  

## dbscan
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN

In [127]:
from sklearn.cluster import DBSCAN
eps = 58
eps = np.percentile(List_of_Frames[0]['ER'], 80)
db = DBSCAN(eps=eps, min_samples = 1).fit(X)
db.labels_

array([ 0,  1,  2,  3,  4,  3,  4,  4,  5,  4,  6,  7,  8,  9, 10, 11, 11,
       11, 11, 12, 13, 11, 11, 14, 11, 14, 15, 11, 11, 16, 11, 11, 11, 11,
       16, 11, 11, 16, 11, 11, 11, 16, 17, 18, 19, 20, 11, 21, 11, 11, 11,
       11], dtype=int64)

In [128]:
labels = db.labels_
print('Estimated number of clusters: ' , len(set(labels)) - (1 if -1 in labels else 0))
print('Estimated number of noise points: ' , list(labels).count(-1))
print('Number of H: ', List_of_Frames[0]['H(J)'].value_counts().count())

Estimated number of clusters:  22
Estimated number of noise points:  0
Number of H:  7


In [129]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(db.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,18
11249,536,2,654.4922,1,1
11250,536,3,654.4922,1,2
11292,536,45,890.9880,2,19
11290,536,43,1324.9020,3,17
11260,536,13,1862.9310,4,8
11252,536,5,2290.8640,5,4
11254,536,7,2290.8640,5,4
11255,536,8,2290.8640,5,4
11257,536,10,2290.8640,5,4


In [130]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.849, S = 0.8, I = 0.545, E = 0.8
Mean: 0.748, compose: 0.555


In [131]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

-0.10312014853912073

Дальше идут методы, считающие известным количество кластеров. Такие методы не рассматриваются в данной работе.

## SpectralClustering
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html#sklearn.cluster.SpectralClustering

In [132]:
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=7, affinity="precomputed", random_state=0).fit(X)
clustering.labels_

array([0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 1,
       3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [133]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']].sort_values('Hlabel')

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11291,536,44,194.7446,0,0
11249,536,2,654.4922,1,0
11250,536,3,654.4922,1,0
11292,536,45,890.9880,2,0
11290,536,43,1324.9020,3,0
11260,536,13,1862.9310,4,0
11252,536,5,2290.8640,5,0
11254,536,7,2290.8640,5,0
11255,536,8,2290.8640,5,5
11257,536,10,2290.8640,5,0


In [134]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.943, S = 0.067, I = 0.143, E = 0.067
Mean: 0.305, compose: 0.053


In [135]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

-0.6343033169337946

## KMeans
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans

In [136]:
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=7, random_state=0).fit(X)
clustering.labels_

array([5, 2, 5, 1, 5, 1, 5, 5, 1, 5, 2, 2, 2, 1, 6, 1, 1, 1, 1, 6, 6, 1,
       1, 6, 1, 6, 3, 1, 1, 6, 6, 1, 6, 1, 6, 1, 1, 6, 1, 1, 1, 6, 4, 0,
       0, 4, 1, 6, 1, 1, 1, 1])

In [137]:
List_of_Frames[k]['Hlabel'] = le.fit_transform(List_of_Frames[k]['H(J)'])
List_of_Frames[k]['labels'] = noise_cluster(clustering.labels_)
List_of_Frames[k][[' num_of_family', ' j', 'H(J)', 'Hlabel', 'labels']]

there is no -1


,num_of_family,j,H(J),Hlabel,labels
11248,536,1,2919.4080,8,5
11249,536,2,654.4922,1,2
11250,536,3,654.4922,1,5
11251,536,4,5842.9680,14,1
11252,536,5,2290.8640,5,5
11253,536,6,5842.9680,14,1
11254,536,7,2290.8640,5,5
11255,536,8,2290.8640,5,5
11256,536,9,5842.9680,14,1
11257,536,10,2290.8640,5,5


In [138]:
CoefPFI(List_of_Frames[k], ' num_of_fam', 'Hlabel', 'labels')

P = 0.869, S = 0.4, I = 0.857, E = 0.339
Mean: 0.616, compose: 0.38


In [139]:
metrics.silhouette_score(X, List_of_Frames[k]['labels'], metric='precomputed')

0.0010128040455083926